<a href="https://colab.research.google.com/github/javadmoradpour/Coursera-machine-learning-project/blob/master/Data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Vaccine uptake project

Our first data source is **Our World in Data**. This data is updated daily and more information about it can be found [here](https://github.com/owid/covid-19-data/tree/master/public/data). In that link, we can find a detailed description of each of the variables. The following table is a summary of important variables included in this data set.

| Metrics | Source | Updated | Countries |
|-------- | -------|---------|-----------|
| Vaccinations | Official data collated by the Our World in Data team |Daily|218|
| Tests & positivity | Official data collated by the Our World in Data team |Weekly|139|
|Hospital & ICU | Official data collated by the Our World in Data team | Weekly | 38 |
|Confirmed cases | JHU CSSE COVID-19 Data | Daily | 196 |
|Confirmed deaths | JHU CSSE COVID-19 Data | Daily | 196 |
|Reproduction rate | Arroyo-Marioli F, Bullano F, Kucinskas S, Rondón-Moreno C | Daily|185|
|Policy responses | Oxford COVID-19 Government Response Tracker | Daily | 186 |
|Other variables of interest | International organizations (UN, World Bank, OECD, IHME…) | Fixed | 241 |

The following lines of codes download this data set from its sources and display the first few rows as an example.

In [ ]:
import pandas as pd
import numpy as np
import requests
import pycountry
from datetime import datetime
import statsmodels.api as sm

import zipfile
from urllib.request import urlopen
import io
import urllib.request

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"

owid=pd.read_csv(url)

Next line of code can show the list of all the available variables in this data set. You need to delete the sign # to see the list. 

In [ ]:
#list(owid.columns)

Having that, we create a data set with static variables that do not change with the date. The following lines of codes create a data set with chosen variables and delete the repetition. The list of the variables can be changed. 

In [ ]:
Data=owid[['iso_code','continent','location','population','population_density','median_age','aged_65_older','aged_70_older','gdp_per_capita','extreme_poverty',
      'hospital_beds_per_thousand','life_expectancy','human_development_index']].drop_duplicates(subset=['location']).reset_index(drop=True)

Data.head()

,iso_code,continent,location,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,3.983543e+07,54.422,18.6,2.581,1.337,1803.987,NaN,0.50,64.83,0.511
1,OWID_AFR,NaN,Africa,1.373486e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALB,Europe,Albania,2.872934e+06,104.871,38.0,13.188,8.643,11803.431,1.1,2.89,78.57,0.795
3,DZA,Africa,Algeria,4.461663e+07,17.348,29.1,6.211,3.857,13913.839,0.5,1.90,76.88,0.748
4,AND,Europe,Andorra,7.735400e+04,163.755,NaN,NaN,NaN,NaN,NaN,NaN,83.73,0.868


As we can see in the data, some of the rows are not for a country, they are for specific regions of the world. So, next, I write a function to check if iso_code corresponds to a country.  Then, using this function, I filter out any row that is not a country.

In [ ]:
def check_if_country(code):
    for co in list(pycountry.countries):
        if code in co.alpha_3:
            return True
    return False

In [ ]:
if_country=[check_if_country(code) for code in Data.iso_code]

Data=Data[if_country].reset_index(drop=True)

The next function finds the value of a given variable at a certain date, for a specified country.

In [ ]:
def find_value_at_fixed_date(country, fixed_date, variable_name):
    
    df=owid[owid.location==country][['date', variable_name]]

    df=df[df['date']<=fixed_date].dropna().reset_index(drop=True)
    
    if df.empty:
        return np.NaN
    
    return df[variable_name].iloc[-1]

Next, we want to define new variables, based on columns that change with the date. To do that, we need to have a starting date for our data. One option is the date when the World Health Organization declared the COVID 19 outbreak a Public Health Emergency of International Concern. This happened on 30 January 2020. [Reference 1](https://www.who.int/news/item/30-01-2020-statement-on-the-second-meeting-of-the-international-health-regulations-(2005)-emergency-committee-regarding-the-outbreak-of-novel-coronavirus-(2019-ncov)). The other option is when the first vaccine has been approved by the health authority in a major country. UK was the first western country in the world to approve a COVID-19 vaccine. UK government approved the Pfizer vaccine on December 2, 2020. [Reference 2](https://www.gov.uk/government/news/uk-medicines-regulator-gives-approval-for-first-uk-covid-19-vaccine) [Reference 3](https://www.theguardian.com/society/2020/dec/02/pfizer-biontech-covid-vaccine-wins-licence-for-use-in-the-uk). We chose the second option. 

Having that, we calculate the total cases and the total number of deaths, before the start of the vaccination campaign.

In [ ]:
start_vaccine_date = '2020-12-02'

variable_list=['total_cases', 'total_deaths']

for variable_name in variable_list:

    Total_value_before_vaccines=[find_value_at_fixed_date(country, start_vaccine_date, variable_name) for country in Data.location]

    Column_name=variable_name+'_at_'+str(start_vaccine_date).replace('-', '_')

    Data[Column_name]=Total_value_before_vaccines

Then we calculate the percentage of people fully vaccinated, for a given set of dates.

In [ ]:
fixed_dates=['2021-03-31', '2021-06-30','2021-09-30', '2021-12-31']

for dt in fixed_dates:

    Percentage_at_fixed_date=[find_value_at_fixed_date(country, dt, 'people_fully_vaccinated_per_hundred') for country in Data.location]

    Column_name='fully_vaccinated_at_'+str(dt).replace('-', '_')

    Data[Column_name]=Percentage_at_fixed_date
    
    Data[Column_name] = Data[Column_name].fillna(0)

Next, we save the data. 

In [ ]:
Data.to_excel("OWID (fixed date).xlsx",index=False)  

Data.to_stata("OWID (fixed date).dta",write_index=False)

Now we add more data from other sources. First added data is a dummy variable, indicating if a country is a member of EMA [Reference 1](https://www.ema.europa.eu/en/partners-networks/eu-partners/eu-member-states/national-competent-authorities-human):

In [ ]:
EMA=pd.read_excel('EMA.xlsx', index_col=None, header=0)[['iso_code','EEA']]

EMA=EMA.rename(columns={"EEA": "EMA"})

Data= Data.merge(EMA, how='left', on='iso_code')

Next we add the data for Corruption Perception Index from Transparency international website. [Reference 1](https://www.transparency.org/en/cpi/2021)

In [ ]:
url = "https://images.transparencycdn.org/images/CPI-2021-Full-Data-Set.zip"

#defining header
header= {'User-Agent':'Chrome/23.0.1271.64 Safari/537.11'}

#the URL where you are requesting at
req = urllib.request.Request(url=url, headers=header) 

zf = zipfile.ZipFile(io.BytesIO(urlopen(req).read()))

xlfile = zf.open(zf.namelist()[1])

CPI = pd.read_excel(xlfile, index_col=None, header=2, sheet_name='CPI Timeseries 2012 - 2021')[['ISO3','CPI score 2020']]

Data= Data.merge(CPI, how='left',left_on='iso_code', right_on='ISO3').drop(columns=['ISO3'])

Here we add the data for vaccine confidence from a recently published paper. [Reference 1](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7607345)

In [ ]:
url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7607345/bin/mmc2.xlsx"

Vaccine_Confidence=pd.read_excel(url, index_col=None, header=0, sheet_name='model_fit_safe')[['country or territory']].drop_duplicates()

sheet_names=['model_fit_safe','model_fit_important', 'model_fit_effective']

for sh_name in sheet_names:

    df=pd.read_excel(url, index_col=None, header=0, sheet_name=sh_name)

    df_columns=list(df.columns)[1:]

    df[df_columns]=df.groupby('country or territory').fillna(method='ffill')

    df= df.groupby('country or territory').tail(1).reset_index(drop=True)[['country or territory', 'mean']]

    df = df.rename(columns={'mean': sh_name}) 

    Vaccine_Confidence=Vaccine_Confidence.merge(df, how='left',on='country or territory')

Next paragraph cleans the vaccine confidence data, before adding it to the main data frame.

In [ ]:
exclude_countries=['Northern Cyprus','Kosovo']
Vaccine_Confidence=Vaccine_Confidence[~Vaccine_Confidence['country or territory'].isin(exclude_countries)].reset_index(drop=True)

Country_Map={'Democratic Republic of the Congo':'Congo, The Democratic Republic of the', 'Republic of Congo':'Republic of the Congo', 
             'Ivory Coast':'Côte d\'Ivoire', 'Swaziland':'Kingdom of Eswatini','UK':'United Kingdom'} 

for i in range(Vaccine_Confidence.shape[0]):
    if Vaccine_Confidence['country or territory'][i] in Country_Map.keys():
        Vaccine_Confidence.loc[i,'country or territory']=Country_Map[Vaccine_Confidence['country or territory'][i]]

        
Vaccine_Confidence['iso_code']=np.nan

for i in range(len(Vaccine_Confidence)):
    count=Vaccine_Confidence.loc[i,'country or territory']
    if count in list(Data.location):
        Vaccine_Confidence.loc[i,'iso_code']=Data[Data.location==count].reset_index().loc[0,'iso_code']
    else:
        Vaccine_Confidence.loc[i,'iso_code']=pycountry.countries.search_fuzzy(count)[0].alpha_3
        
Data= Data.merge(Vaccine_Confidence, how='left',on='iso_code').drop(columns=['country or territory'])   

Next we add the data for education from world bank. [Reference 1](https://data.worldbank.org/topic/4)

In [ ]:
url = "https://api.worldbank.org/v2/en/topic/4?downloadformat=excel"

Education_df=pd.read_excel(url, index_col=None, header=3, sheet_name='Data').drop(columns=['Indicator Code'])

Education_Variables=['School enrollment, primary (% net)','School enrollment, secondary (% net)',
                     'School enrollment, tertiary (% gross)','Literacy rate, adult total (% of people ages 15 and above)']

Education_df=Education_df[Education_df['Indicator Name'].isin(Education_Variables)].reset_index(drop=True)

Education_df=Education_df.fillna(method='ffill',axis=1)[['Country Name','Country Code','Indicator Name','2020']]

Education_df['2020']=pd.to_numeric(Education_df['2020'],errors='coerce')


Education=Education_df[['Country Name','Country Code']].drop_duplicates().reset_index(drop=True)

for var in Education_Variables:
    Education[var]=Education_df[Education_df['Indicator Name']==var].reset_index(drop=True)['2020']
    
Data= Data.merge(Education, how='left',left_on='iso_code', right_on='Country Code').drop(columns=['Country Code','Country Name'])

Next, we prepare our data for running the regression. To do that, we select the variable we want to use for the analysis, delete all the rows with missing values, and define our dependent and independent variables. 

In [ ]:
#dependent_variable='fully_vaccinated_at_2021_06_30'

dependent_variable='fully_vaccinated_at_2021_12_31'

#dependent_variable='fully_vaccinated_at_2021_09_30'

data=Data[[dependent_variable,
           'gdp_per_capita',
           'aged_65_older',
           'population', 
           'population_density',
           'CPI score 2020',
           'model_fit_safe',
           #'School enrollment, tertiary (% gross)',
           #'School enrollment, primary (% net)',
           'EMA',
           'total_deaths_at_2020_12_02'
          ]].dropna().reset_index(drop=True)

loged_variables=['gdp_per_capita',
                 'population',
                 'total_deaths_at_2020_12_02',
                 'population_density',
                ]

for var in loged_variables:
    data[var]=np.log(data[var])
    data.rename(columns = {var:'log_'+var}, inplace = True)


Y= data[dependent_variable]

X=data.drop(columns=[dependent_variable])



mod = sm.OLS(Y, X)

res = mod.fit()

print(res.summary())

                                       OLS Regression Results                                      
Dep. Variable:     fully_vaccinated_at_2021_12_31   R-squared (uncentered):                   0.905
Model:                                        OLS   Adj. R-squared (uncentered):              0.899
Method:                             Least Squares   F-statistic:                              155.6
Date:                            Mon, 02 May 2022   Prob (F-statistic):                    4.81e-63
Time:                                    17:02:32   Log-Likelihood:                         -579.25
No. Observations:                             139   AIC:                                      1175.
Df Residuals:                                 131   BIC:                                      1198.
Df Model:                                       8                                                  
Covariance Type:                        nonrobust                                                  


In [ ]:
list(Data.columns)

['iso_code',
 'continent',
 'location',
 'population',
 'population_density',
 'median_age',
 'aged_65_older',
 'aged_70_older',
 'gdp_per_capita',
 'extreme_poverty',
 'hospital_beds_per_thousand',
 'life_expectancy',
 'human_development_index',
 'total_cases_at_2020_12_02',
 'total_deaths_at_2020_12_02',
 'fully_vaccinated_at_2021_03_31',
 'fully_vaccinated_at_2021_06_30',
 'fully_vaccinated_at_2021_09_30',
 'fully_vaccinated_at_2021_12_31',
 'EMA',
 'CPI score 2020',
 'model_fit_safe',
 'model_fit_important',
 'model_fit_effective',
 'School enrollment, primary (% net)',
 'School enrollment, secondary (% net)',
 'School enrollment, tertiary (% gross)',
 'Literacy rate, adult total (% of people ages 15 and above)']

In [ ]:
Data.rename(columns = {'CPI score 2020':'CPI_score_2020',
                       'School enrollment, primary (% net)':'School_enrollment_primary_net',
                       'School enrollment, secondary (% net)':'School_enrollment_secondary_net',
                       'School enrollment, tertiary (% gross)':'School_enrollment_tertiary_gross',
                       'Literacy rate, adult total (% of people ages 15 and above)':'Literacy_rate_adult'
                      }, inplace = True)


Data.to_excel("Data.xlsx",index=False)  

Data.to_stata("Data.dta",write_index=False)